# Challenge

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

1. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
2. What metric would you report for this dataset?
3. What is its value?

In [1]:
# Import libraries
import pandas as pd
import plotly.express as px
from pathlib import Path

# Check and Prepare Data for Analysis

In [2]:
# Read in csv with 'created_at' column as index
file_path = Path('Resources/Shopify-Data-Science-Challenge.csv')
df = pd.read_csv(file_path, index_col='created_at', infer_datetime_format=True, parse_dates=True)
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-13 12:36:56,1,53,746,224,2,cash
2017-03-03 17:38:52,2,92,925,90,1,cash
2017-03-14 04:23:56,3,44,861,144,1,cash
2017-03-26 12:43:37,4,18,935,156,1,credit_card
2017-03-01 04:35:11,5,18,883,156,1,credit_card


In [3]:
# Reset index and drop times from datetime 'created_at' column
df.reset_index(inplace=True)
df['created_at'] = df['created_at'].dt.date

In [4]:
# Sort DataFrame by date
df = df.sort_values(by='created_at')

In [5]:
# Set 'created_at' column back to index
df.set_index('created_at', inplace=True)

In [6]:
# Check DataFrame
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-01,1144,3,946,296,2,debit
2017-03-01,1611,25,941,260,2,cash
2017-03-01,1609,18,864,156,1,credit_card
2017-03-01,3147,28,750,492,3,credit_card
2017-03-01,3153,46,727,166,1,cash


In [7]:
# Check nulls
df.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
dtype: int64

In [8]:
# Check duplicates
df.duplicated().sum()

0

In [9]:
# Check Average Order Value (AOV)
df['order_amount'].mean()

3145.128

In [10]:
# Check max date
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-01,1144,3,946,296,2,debit
2017-03-01,1611,25,941,260,2,cash
2017-03-01,1609,18,864,156,1,credit_card
2017-03-01,3147,28,750,492,3,credit_card
2017-03-01,3153,46,727,166,1,cash


In [11]:
# Check min date
df.tail()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-30,843,39,839,402,3,credit_card
2017-03-30,3989,27,714,169,1,credit_card
2017-03-30,883,23,947,624,4,debit
2017-03-30,955,37,776,568,4,credit_card
2017-03-30,3644,41,764,472,4,debit


In [12]:
#Check number of orders
df.count()

order_id          5000
shop_id           5000
user_id           5000
order_amount      5000
total_items       5000
payment_method    5000
dtype: int64

In [13]:
# Check total_items summary
df['total_items'].describe()

count    5000.00000
mean        8.78720
std       116.32032
min         1.00000
25%         1.00000
50%         2.00000
75%         3.00000
max      2000.00000
Name: total_items, dtype: float64

In [14]:
# Check order_amount summary
df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

# Extreme Outliers Observations

As demonstrated by the boxplots below:

1. Items Outliers - We have 12 total extreme outliers with 2000 items purchased. No other extreme outliers are observed.
2. Order Amount Outliers - We have the same number of total extreme outliers. There is some greater variability in order amount outliers when compared with the items outliers. Overall, this is not concerning since shoe prices can vary by a fair margin.

## Next Steps

1. Items Outliers - Validate if 12 extreme outliers observed in the boxplots is accurate with the dataset or if there are duplicate orders with the same amount.
2. Validate whether the extreme outlier is the same shop_id or same user_id to better understand the source of these outliers.

In [15]:
# Boxplot total_items outliers
items_outliers = px.box(
    df,
    x=df.index,
    y='total_items',
    title='Total Items Outliers Analysis',
    labels={'created_at': 'Date', 'total_items': 'Total Items'},
    color=df.index,
    boxmode='overlay'
    )

items_outliers

In [16]:
# Boxplot order_amount outliers
amount_outliers = px.box(
    df,
    x=df.index,
    y='order_amount',
    title='Order Amount Outliers Analysis',
    labels={'created_at': 'Date', 'order_amount': 'Order Amount'},
    color=df.index,
    boxmode='overlay'
    )

amount_outliers

# Identifying the Source of Extreme Outliers

As shown below, shop_id 42 and user_id 607 are consistently the extreme outlier in the dataset. 

## Next Step

Identify if there are other orders for shop_id 42 and user_id 607

In [17]:
# Check where total_items == 2000
df.loc[df['total_items'] == 2000]

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-02,521,42,607,704000,2000,credit_card
2017-03-02,4647,42,607,704000,2000,credit_card
2017-03-04,61,42,607,704000,2000,credit_card
2017-03-07,2298,42,607,704000,2000,credit_card
2017-03-07,16,42,607,704000,2000,credit_card
2017-03-11,1437,42,607,704000,2000,credit_card
2017-03-12,2154,42,607,704000,2000,credit_card
2017-03-15,1363,42,607,704000,2000,credit_card
2017-03-17,1603,42,607,704000,2000,credit_card


In [18]:
# Check observations count
df.loc[df['total_items'] == 2000].count()

order_id          17
shop_id           17
user_id           17
order_amount      17
total_items       17
payment_method    17
dtype: int64

In [19]:
#Check shop_id 42 for other orders
df.loc[df['shop_id'] == 42]

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-01,4422,42,736,704,2,credit_card
2017-03-01,2019,42,739,352,1,debit
2017-03-01,2492,42,868,704,2,debit
2017-03-02,521,42,607,704000,2000,credit_card
2017-03-02,4647,42,607,704000,2000,credit_card
2017-03-03,2988,42,819,1056,3,cash
2017-03-04,410,42,904,704,2,credit_card
2017-03-04,61,42,607,704000,2000,credit_card
2017-03-04,4232,42,962,352,1,cash


In [20]:
# Check summary
df.loc[df['shop_id'] == 42].describe()

,order_id,shop_id,user_id,order_amount,total_items
count,51.000000,51.0,51.000000,51.000000,51.000000
mean,2441.921569,42.0,758.588235,235101.490196,667.901961
std,1484.456801,0.0,125.993044,334860.641587,951.308641
min,16.000000,42.0,607.000000,352.000000,1.000000
25%,1366.500000,42.0,607.000000,352.000000,1.000000
50%,2154.000000,42.0,770.000000,704.000000,2.000000
75%,3801.000000,42.0,863.500000,704000.000000,2000.000000
max,4883.000000,42.0,975.000000,704000.000000,2000.000000


In [21]:
# Check user_id 607 for other orders
df.loc[df['user_id'] == 607]

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-02,521,42,607,704000,2000,credit_card
2017-03-02,4647,42,607,704000,2000,credit_card
2017-03-04,61,42,607,704000,2000,credit_card
2017-03-07,2298,42,607,704000,2000,credit_card
2017-03-07,16,42,607,704000,2000,credit_card
2017-03-11,1437,42,607,704000,2000,credit_card
2017-03-12,2154,42,607,704000,2000,credit_card
2017-03-15,1363,42,607,704000,2000,credit_card
2017-03-17,1603,42,607,704000,2000,credit_card


In [22]:
# Check order_amount is always from user_id 607
df.loc[df['order_amount'] == 704000]

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-02,521,42,607,704000,2000,credit_card
2017-03-02,4647,42,607,704000,2000,credit_card
2017-03-04,61,42,607,704000,2000,credit_card
2017-03-07,2298,42,607,704000,2000,credit_card
2017-03-07,16,42,607,704000,2000,credit_card
2017-03-11,1437,42,607,704000,2000,credit_card
2017-03-12,2154,42,607,704000,2000,credit_card
2017-03-15,1363,42,607,704000,2000,credit_card
2017-03-17,1603,42,607,704000,2000,credit_card


In [23]:
# Check user_id 607 order count
df.loc[df['user_id'] == 607].count()

order_id          17
shop_id           17
user_id           17
order_amount      17
total_items       17
payment_method    17
dtype: int64

# Calculate Outlier-Adjusted AOV Metric

From our analysis, shop_id 42 has other user_ids submitting orders that do not fall into the extreme outlier category. Every order from user_id 607 is an extreme outlier.

## Next Steps

1. Remove user_id 607 orders from dataset.
2. Calculate outlier-adjusted AOV metric

In [24]:
# Remove user_id 607 from DataFrame
new_df = df[df.user_id != 607]

In [25]:
# Check user_id 607 is removed from DataFrame
new_df.loc[new_df['user_id'] == 607]

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,


In [26]:
# Calculate AOV with new DataFrame
new_df['order_amount'].mean()

754.0919125025085

# Summary Analysis - Challenge Questions Response

1. <p>Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.<br>
    a. In the dataset provided, there are 17 extreme outlier observations with a value of 2000. A better way to evaluate this data is to remove the extreme outliers as demonstrated above.
    </p> 
2. <p>What metric would you report for this dataset?<br>
    b. The metric to report for this dataset is an outlier-adjusted AOV metric by removing the extreme outliers from the AOV calculation.
    </p>
3. <p>What is its value?<br>
    c. The outlier-adjusted AOV metric is $754.09 compared to the naive AOV of $3145.13.</p>
